# Topic ideas

---

Student name: Priscila Strömsdörfer - ps178@hdm-stuttgart.de

---

## New Topic Idea -
## Topic idea 1: Analyse der Lieferzuverlässigkeit anhand von Lieferplänen

Ziel des Projekts ist es, die Lieferzuverlässigkeit im Logistik-Umfeld von Mercedes-Benz zu analysieren. Dabei sollen Überlieferungen und Unterlieferungen identifiziert und bewertet werden, um die Effizienz der Lieferkette zu verbessern.

### Data source

*Der Datensatz stammt aus den internen SAP-basierten Logistiksystemen der Mercedes-Benz AG. Die Analyse basiert auf den LIPS und LIKP (Lieferpositionsdaten) und auf EKPO und EKKO (Bestelldaten). Die Daten werden verknüpft um geplante und tatsächliche Lieferungen zu vergleichen.* 

Berechnung der Lieferuzuverlässigkeit: Definieren von Metriken zur Bewertung der Lieferzuverlässigkeit, z.B.:
- Liefertermintreue: Anteil der Lieferungen, die pünktlich (innerhalb eines definierten Zeitfensters) erfolgen.
- Mengenabweichung: Differenz zwischen bestellten und tatsächlich gelieferten Mengen.

### Data characteristics
| **Kategorie**              | **Datenquelle (SAP-Tabellen)** | **Merkmale**                                   | **Beschreibung**                                                                 |
|----------------------------|--------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| **Bestelldaten**           | EKPO, EKKO                    | Bestellnummer, Bestelldatum, geplantes Lieferdatum | Informationen zu Bestellungen, wie geplante Lieferdaten und Mengen.             |
| **Lieferdaten**            | LIPS, LIKP                    | Liefernummer, Lieferdatum, gelieferte Menge    | Details zu tatsächlichen Lieferungen, wie Liefertermine und gelieferte Mengen.  |
| **Abweichungsanalysen**    | Kombination aus allen         | Geplante vs. tatsächliche Lieferdaten          | Daten für die Berechnung von Liefertermintreue und Mengenabweichungen.          |
| **Lieferanteninformationen** | Lieferantenstamm             | Lieferantennummer, Name, Kategorie             | Informationen über die Lieferanten, um Analysen nach Lieferantengruppen zu ermöglichen. |
| **Produktinformationen**   | Materialstamm                | Produktnummer, Kategorie                       | Informationen über die Produkte zur Identifikation von Produktkategorien.       |

1. **Zeitraum**:
   - Daten über einen Zeitraum von **12 Monaten**.
   - Aggregation auf **Monatsebene** (z. B. durchschnittliche Termintreue oder Mengenabweichung pro Monat).

2. **Granularität**:
   - **Originaldaten** auf Positionsebene (Lieferpositionen und Bestellpositionen).
   - **Aggregierte Metriken** auf Monatsebene:
     - **Liefertermintreue**: Anteil pünktlicher Lieferungen pro Monat.
     - **Mengenabweichung**: Durchschnittliche Abweichung der gelieferten Mengen pro Monat.

3. **Datenvolumen**:
   - Je nach Umfang der Liefer- und Bestelldaten **tausende Einträge**.
   - Nach Aggregation stark reduziert (z. B. **12 Einträge** pro Lieferant oder Produktkategorie).

### Research question
1.	Hauptfokus auf Termintreue:
- Welche Faktoren beeinflussen die Liefertermintreue und wie können Abweichungen zwischen geplanten und tatsächlichen Lieferterminen reduziert werden?

    a. Deskriptive Statistik:
        - Mittelwert, Median, Modus, Standardabweichung.

    b. Zeitreihenmodelle (für Trends und Vorhersagen):
        - Exponentielle Glättung: Einfache Methode für kurzfristige Vorhersagen.

    c. Regressionsmodelle:
        - Lineare Regression zur Analyse der Beziehung zwischen geplanten und tatsächlichen Lieferterminen oder Mengen.

    d. Klassifikationsmodelle:
        - Logistische Regression: Vorhersage, ob eine Lieferung pünktlich oder verspätet ist.

### Overview of data

*Beispieldaten:*

In [3]:
import pandas as pd

# Beispiel-Datensätze mit relevanten Datenfeldern erstellen
example_data = {
    "Monat": ["January 2023", "February 2023", "March 2023", "April 2023", "May 2023"],
    "Lieferant": ["Lieferant A", "Lieferant B", "Lieferant C", "Lieferant A", "Lieferant B"],
    "Produktkategorie": ["Kategorie X", "Kategorie Y", "Kategorie Z", "Kategorie X", "Kategorie Y"],
    "Geplantes Lieferdatum": pd.to_datetime(
        ["2023-01-10", "2023-02-15", "2023-03-20", "2023-04-25", "2023-05-30"]
    ),
    "Tatsächliches Lieferdatum": pd.to_datetime(
        ["2023-01-12", "2023-02-13", "2023-03-22", "2023-04-23", "2023-05-29"]
    ),
    "Bestellte Menge": [100, 200, 150, 300, 250],
    "Gelieferte Menge": [98, 210, 150, 295, 255],
    "Durchschnittliche Liefertermintreue (%)": [90.5, 85.3, 92.0, 96.8, 89.4],
    "Durchschnittliche Mengenabweichung": [-2, 10, 0, -5, 5],
}

# DataFrame erstellen
example_df = pd.DataFrame(example_data)

example_df.head()

,Monat,Lieferant,Produktkategorie,Geplantes Lieferdatum,Tatsächliches Lieferdatum,Bestellte Menge,Gelieferte Menge,Durchschnittliche Liefertermintreue (%),Durchschnittliche Mengenabweichung
0,January 2023,Lieferant A,Kategorie X,2023-01-10,2023-01-12,100,98,90.5,-2
1,February 2023,Lieferant B,Kategorie Y,2023-02-15,2023-02-13,200,210,85.3,10
2,March 2023,Lieferant C,Kategorie Z,2023-03-20,2023-03-22,150,150,92.0,0
3,April 2023,Lieferant A,Kategorie X,2023-04-25,2023-04-23,300,295,96.8,-5
4,May 2023,Lieferant B,Kategorie Y,2023-05-30,2023-05-29,250,255,89.4,5
